# ハミルトニアン (N=2)

![hamiltonian(3a,N=2)](./Figures/3a_N2.png)

\begin{align}
\mathcal{H} = \sum^{1}_{i=0}\mathcal{H}_i + \sum_{(i, j)=(2, 4), (0, 6)}V_{ij}
\end{align}

\begin{align}
\mathcal{H}_i &= \sum_{(j, k) \in E}\sum_{A=X,Y,Z}A_{j}A_{k} \\
V_{ij} &= \sum_{A=X,Y,Z}A_{i}A_{j}
\end{align}

上の図で黒い線で繋がった格子が部分系$\mathcal{H}_i$で赤い線が相互作用$V_{ij}$をそれぞれ表している。  

In [1]:
from qulacs.gate import Pauli
from qulacs.gate import Identity, X, Y, Z, to_matrix_gate
import numpy as np

http://docs.qulacs.org/ja/latest/guide/2.0_python_advanced.html

# 厳密対角化で解く場合
ハミルトニアンを分割せずに解く場合

In [2]:
n_bits = 8
h_dim = pow(2, n_bits)
H = np.zeros((h_dim , h_dim )).astype(complex)
E = [(0, 1), (1, 2), (2, 3), (3, 0), (0, 2)]
for s in range(2):
    for (i, j) in E:
        target_list = [b for b in range(n_bits)]
        for k in range(1, 4):
            pauli_index = [0 for b in range(n_bits)]
            pauli_index[i+s*n_bits//2] = k
            pauli_index[j+s*n_bits//2] = k
            gate = Pauli(target_list, pauli_index)
            H += gate.get_matrix()
(i, j) = (2, 4)
target_list = [b for b in range(n_bits)]
for k in range(1, 4):
    pauli_index = [0 for b in range(n_bits)]
    pauli_index[i] = k
    pauli_index[j] = k
    gate = Pauli(target_list, pauli_index)
    H += gate.get_matrix()

### 対角化の実行

In [3]:
Evals, Evecs = np.linalg.eig(H)
State = np.transpose(Evecs)
gs = State[np.argmin(Evals)]
print("Ground State Energy: {}".format(min(Evals)))

Ground State Energy: (-14.464101615137706-1.6875949499599427e-24j)


# 有効ハミルトニアンを対角化で解く場合
VQEの代わりに行列を直接対角化して基底状態を求める。  
`gs`に基底状態のベクトルが格納されている。 

## 部分系の対角化

In [4]:
n_bits = 4
h_dim = pow(2, n_bits)
H = np.zeros((h_dim , h_dim )).astype(complex)
E = [(0, 1), (1, 2), (2, 3), (3, 0), (0, 2)]
for (i, j) in E:
    target_list = [b for b in range(n_bits)]
    for k in range(1, 4):
        pauli_index = [0 for b in range(n_bits)]
        pauli_index[i] = k
        pauli_index[j] = k
        gate = Pauli(target_list, pauli_index)
        H += gate.get_matrix()

### 対角化の実行

In [5]:
Evals, Evecs = np.linalg.eig(H)
State = np.transpose(Evecs)
gs = State[np.argmin(Evals)]
print("Ground State Energy: {}".format(min(Evals)))

Ground State Energy: (-7+0j)


## 局所基底の作成
`psi`はWを`gs`に作用させて作った基底である。  
この基底をグラムシュミットの直交化法により正規直交基底に変換したものが`psi_norm`である。

In [6]:
W = []
# Identity Matrix
target_list = [b for b in range(n_bits)]
pauli_index = [0 for b in range(n_bits)]
W.append(Pauli(target_list, pauli_index).get_matrix())
for i in [0, 2]:
    target_list = [b for b in range(n_bits)]
    for j in range(1, 4):
        pauli_index = [0 for b in range(n_bits)]
        pauli_index[i] = j
        W.append(Pauli(target_list, pauli_index).get_matrix())
K = len(W)

In [7]:
psi = []
for i in range(K):
    psi.append(W[i] @ gs)
h_dim = pow(2, n_bits)
psi_norm = np.zeros((K, h_dim)).astype(complex)
for i in range(K):
    psi_norm[i]+= psi[i]
    for j in range(i):
        psi_norm[i] -= (psi_norm[j].conjugate() @ psi[i]) * psi_norm[j]
    psi_norm[i] *= 1.0/(np.sqrt(psi_norm[i].conjugate() @ psi_norm[i]))

In [8]:
WW = np.zeros((K, K)).astype(complex)
for i in range(K):
    for j in range(K):
        WW[i][j] = psi[i].conjugate() @ psi[j]

### 正規直行化されているかの確認

In [9]:
norm = np.zeros((K, K))
for i in range(K):
    for j in range(K):
        coeff = psi_norm[i].conjugate() @ psi_norm[j]
        coeff = abs(coeff)
        if coeff <1e-5:
            coeff = 0
        norm[i, j] = coeff
print(norm)

[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]]


## 部分系の有効ハミルトニアンの作成
`psi_norm`で`H`の行列要素を計算したものが`H_eff_i`である。

In [10]:
H_eff_i = np.zeros((K, K)).astype(complex)
H_bar_i = np.zeros((K, K)).astype(complex)
for i in range(K):
    for j in range(K):
        #H_eff_i[i, j] = np.dot(psi_norm[i].conjugate(), np.dot(H, psi_norm[j]))
        H_eff_i[i, j] = psi_norm[i].conjugate() @ H @ psi_norm[j]
        #H_bar_i[i, j] = np.dot(psi[i].conjugate(), np.dot(H, psi[j]))
        H_bar_i[i, j] = psi[i].conjugate() @ H @ psi[j]

### 対角化の実行

In [11]:
Evals, Evecs = np.linalg.eig(H_bar_i)
State = np.transpose(Evecs)
gs = State[np.argmin(Evals)]
print("Ground State Energy: {}".format(min(Evals)))

Ground State Energy: (-7+0j)


## 相互作用の有効ハミルトニアン
`psi_norm`で`W`の行列要素を計算したものが`Vk`に格納されている。  
これを元に相互作用の有効ハミルトニアン`V_eff`を計算している。

In [12]:
WVW = np.zeros((K-1, K, K)).astype(complex)
WVW_ = np.zeros((K-1, K, K)).astype(complex)
for k in range(K-1):
    for i in range(K):
        for j in range(K):
            WVW[k][i][j] = np.dot(psi_norm[i].conjugate(), np.dot(W[k+1], psi_norm[j]))
            WVW_[k][i][j] = np.dot(psi[i].conjugate(),  np.dot(W[k+1], psi[j]))
V_eff = np.zeros((K, K, K, K)).astype(complex)
for i in range(K):
    for j in range(K):
        for k in range(K):
            for l in range(K):
                coeff = 0
                for m in range(K//2):
                    coeff += WVW[m][i][j] * WVW[m+K//2][k][l]
                    #coeff += WVW[m+K//2][i][j] * WVW[m][k][l]
                V_eff[i][j][k][l] = coeff

## 全体の有効ハミルトニアンの作成
2つの部分系をまとめると必要な基底はK*K個になる。  
この基底を使って全体の有効ハミルトニアンの行列要素`H_eff`を計算する。

In [13]:
local_basis = [(i, j) for i in range(K) for j in range(K)]
H_eff = np.zeros((K*K, K*K)).astype(complex)
for i in range(K*K):
    for j in range(K*K):
        (k, l) = local_basis[i]
        (m, n) = local_basis[j]
        delta1 = (k==m)
        delta2 = (l==n)
        H_eff[i, j] = H_eff_i[k, m]*delta2 + H_eff_i[l, n]*delta1 + V_eff[k][m][l][n]

### 対角化の実行

In [14]:
Evals, Evecs = np.linalg.eig(H_eff)
State = np.transpose(Evecs)
gs = State[np.argmin(Evals)]
print("Ground State Energy: {}".format(min(Evals)))

Ground State Energy: (-14.464101615137757+0j)
